<a href="https://colab.research.google.com/github/Martinaeht/Irony_Detection_roberta_bertweet/blob/master/Irony_Detection_MT_roBERTa_manual_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IRONY DETECTION

**Dataset:** https://huggingface.co/datasets/cardiffnlp/tweet_eval

**Model:** https://huggingface.co/FacebookAI/roberta-base



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate --upgrade
!pip install optuna
!pip install emoji==0.6.0


In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, RobertaModel, RobertaForSequenceClassification
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, set_seed
from transformers import AutoModelForMaskedLM
from pprint import pprint
import numpy as np
import evaluate
import torch
from torch.utils.tensorboard import SummaryWriter
import os
import optuna
import json
from sklearn.metrics import precision_score, recall_score, f1_score

set_seed(333)

irony_dataset = load_dataset("cardiffnlp/tweet_eval", "irony")

train_dataset = irony_dataset['train']
validation_dataset = irony_dataset['validation']
test_dataset = irony_dataset['test']

train_dataset_print = train_dataset[:10]

for label, text in zip(train_dataset_print['label'], train_dataset_print['text']):
    print(label, text)

small_irony_dataset = DatasetDict(
    train = train_dataset.shuffle(seed=333).select(range(640)),
    val = validation_dataset.shuffle(seed=333).select(range(160)),
    test = test_dataset.shuffle(seed=333).select(range(160)))


1 seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life
0 look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. 💕🎵
1 Now I remember why I buy books online @user #servicewithasmile
1 @user @user So is he banded from wearing the clothes?  #Karma
1 Just found out there are Etch A Sketch apps.  #oldschool #notoldschool
1 Hey what do you know, one of the witnesses supporting Darren Wilson's story lied! And is racist! Mind blown!
0 @user on stage at #flzjingleball at the @user in #Tampa #iheartradio
1 You know it's going to be a great day when you're Garmin resets itself and you spill some cinnamon down yourself  #slowclap
1 Halfway thorough my workday ... Woooo
1 Would like to thank my nephew for giving me his horrible cold & sore throat etc.. Much appreciated!


# roBERTa

https://huggingface.co/docs/transformers/main/en/model_doc/roberta

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
#tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

max_length = 128

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",  # Pad to max_length
        truncation=True,       # Truncate if longer
        max_length=max_length  # Set max length
    )

small_tokenized_dataset = small_irony_dataset.map(tokenize_function, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

#model = RobertaModel.from_pretrained("FacebookAI/roberta-base", num_labels=2)
#model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion"
model = RobertaForSequenceClassification.from_pretrained("FacebookAI/roberta-base", num_labels=2)

accuracy = evaluate.load("accuracy")


Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# roBERTa manual finetuning

In [ ]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
#  return accuracy.compute(predictions=predictions, references=labels)
  accuracy = (predictions == labels).mean()
  return {"accuracy": accuracy}

In [ ]:

arguments = TrainingArguments(
    output_dir="sample_cl_trainer_rob_man",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=8,
    num_train_epochs= 5,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate= 2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none',
    seed=333
)


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [ ]:
# test:
print(small_tokenized_dataset['train'][2])


{'text': '@user @user |Yo boy! You guys are know to keep the party mood going. |#EzoneGoaDiaries.', 'label': 0, 'input_ids': [0, 1039, 12105, 787, 12105, 1721, 33543, 2143, 328, 370, 1669, 32, 216, 7, 489, 5, 537, 6711, 164, 4, 1721, 10431, 717, 13930, 11478, 102, 495, 41608, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.680800,0.691971,0.512500
2,0.595000,0.657639,0.612500
3,0.488800,0.778065,0.600000
4,0.287100,0.758177,0.631250
5,0.264400,0.804957,0.618750


TrainOutput(global_step=200, training_loss=0.4819439142942429, metrics={'train_runtime': 5160.7609, 'train_samples_per_second': 0.62, 'train_steps_per_second': 0.039, 'total_flos': 210488844288000.0, 'train_loss': 0.4819439142942429, 'epoch': 5.0})

In [ ]:
test_str1 = "@user @user @user @user I'm off to visit great-nephew very ill, only 20 fair"
test_str2 = "Halfway thorough my workday ... Woooo"

# 1 = irony
# 0 = non_irony

fine_tuned_model = RobertaForSequenceClassification.from_pretrained("sample_cl_trainer_rob_man/checkpoint-160")
model_inputs = tokenizer(test_str2, return_tensors="pt")
prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
print(["non-irony", "irony"][prediction])

irony


In [ ]:
save_directory = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_manually_finetuned_640"

tokenizer.save_pretrained(save_directory)

model.save_pretrained(save_directory)

# roBERTa - testing the finetuned model

In [ ]:

output_dir_test = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_testing_man_finetuned_model_640"
if not os.path.exists(output_dir_test):
  os.makedirs(output_dir_test)

for example in small_tokenized_dataset['test']:
  text = example['text']
  model_inputs = tokenizer(text, return_tensors="pt")
  prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
  predicted_label = ["non-irony", "irony"][prediction]
  print(f"Text: {text}\nPredicted Label: {predicted_label}\n")


arguments_test_set = TrainingArguments(
    output_dir="results_test_set_640",
    per_device_eval_batch_size=16,
    report_to='none',
    seed=333
)

trainer_test_set = Trainer(
    model=fine_tuned_model,
    args=arguments_test_set,
    eval_dataset=small_tokenized_dataset['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

test_results = trainer_test_set.evaluate(eval_dataset=small_tokenized_dataset['test'])
print(f"Test Accuracy: {test_results['eval_accuracy']}")

#output: Test Accuracy: 0.6625
#compared to training accuracy: 0.631250 in best epoch (4/cp160)


Text: #AdvancedWarfare is so much better than any FPS released in 2014 XD
Predicted Label: irony

Text: @user @user @user |*Everything is crossed*|#ChristmasGift |#BassFace
Predicted Label: non-irony

Text: Laborers in the San Joaquin Valley, one of the most productive farming regions in the world, suffer from food insecurity. #irony #poverty
Predicted Label: irony

Text: Pips who drop out of school n have Kirya as 'eir role model tht ey wana sing too..guy spix Portuguese,French,english...#Not a school dropout
Predicted Label: non-irony

Text: I really want to see Into The Woods!
Predicted Label: irony

Text: #Irony: al jazeera is pro Anti - #GamerGate because feminism, or something:
Predicted Label: irony

Text: Sleep. SLEEP YOU STUPID IDIOT I WANNA LEAVE MY ROOM
Predicted Label: irony

Text: Some days are just better than others..
Predicted Label: irony

Text: Christmas alone😊 how nice #not
Predicted Label: irony

Text: Read at the news '#Obama: #The #American #way #of #life #is #not

Test Accuracy: 0.6625


In [ ]:
# F1 score

fine_tuned_model = RobertaForSequenceClassification.from_pretrained("sample_cl_trainer_rob_man/checkpoint-160")

def compute_f1_score(model, tokenizer, dataset):
    all_labels = []
    all_predictions = []

    for example in dataset:
        text = example['text']
        label = example['label']
        model_inputs = tokenizer(text, return_tensors="pt")
        logits = model(**model_inputs).logits
        prediction = torch.argmax(logits, dim=-1).item()  # Get single prediction

        all_labels.append(label)
        all_predictions.append(prediction)

    precision = precision_score(all_labels, all_predictions, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_predictions, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_predictions, average='weighted', zero_division=0)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

print(compute_f1_score(fine_tuned_model, tokenizer, small_tokenized_dataset['test']))


{'precision': 0.6924204849629979, 'recall': 0.6625, 'f1': 0.664826127819549}


worst epoch

In [ ]:
# worst epoch

output_dir_test_worstep = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_testing_finetuned_model_640_worstep_cp40"
if not os.path.exists(output_dir_test_worstep):
  os.makedirs(output_dir_test_worstep)


# adapted checkpoint
fine_tuned_model_worstep = RobertaForSequenceClassification.from_pretrained("sample_cl_trainer_rob_man/checkpoint-40", num_labels=2)


for example in small_tokenized_dataset['test']:
  text = example['text']
  model_inputs = tokenizer(text, return_tensors="pt")
  prediction = torch.argmax(fine_tuned_model_worstep(**model_inputs).logits)
  predicted_label = ["non-irony", "irony"][prediction]
  print(f"Text: {text}\nPredicted Label: {predicted_label}\n")


arguments_test_set = TrainingArguments(
    output_dir="results_test_set_640_worstep",
    per_device_eval_batch_size=16,
    report_to='none',
    seed=333
)

trainer_test_set = Trainer(
    model=fine_tuned_model_worstep,
    args=arguments_test_set,
    eval_dataset=small_tokenized_dataset['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

test_results_worstep = trainer_test_set.evaluate(eval_dataset=small_tokenized_dataset['test'])
print(f"Test Accuracy: {test_results_worstep['eval_accuracy']}")


#output: Test Accuracy: 0.55
#compared to training accuracy: 0.512500


Text: #AdvancedWarfare is so much better than any FPS released in 2014 XD
Predicted Label: irony

Text: @user @user @user |*Everything is crossed*|#ChristmasGift |#BassFace
Predicted Label: non-irony

Text: Laborers in the San Joaquin Valley, one of the most productive farming regions in the world, suffer from food insecurity. #irony #poverty
Predicted Label: irony

Text: Pips who drop out of school n have Kirya as 'eir role model tht ey wana sing too..guy spix Portuguese,French,english...#Not a school dropout
Predicted Label: non-irony

Text: I really want to see Into The Woods!
Predicted Label: irony

Text: #Irony: al jazeera is pro Anti - #GamerGate because feminism, or something:
Predicted Label: irony

Text: Sleep. SLEEP YOU STUPID IDIOT I WANNA LEAVE MY ROOM
Predicted Label: irony

Text: Some days are just better than others..
Predicted Label: irony

Text: Christmas alone😊 how nice #not
Predicted Label: irony

Text: Read at the news '#Obama: #The #American #way #of #life #is #not

Test Accuracy: 0.55


# roBERTa Visualization

https://projector.tensorflow.org/

In [ ]:
# Tensors on different devices --> solution:

# Determine the device (CUDA or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the correct device
fine_tuned_model.to(device)

# Tokenize the input text and move it to the same device
model_inputs = tokenizer(small_tokenized_dataset['test']['text'], padding=True, truncation=True, return_tensors='pt')
model_inputs = {key: value.to(device) for key, value in model_inputs.items()}

outputs = fine_tuned_model(**model_inputs, output_hidden_states=True)


In [ ]:
# best epoch

path = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_results_vis_man_640_cp160"
if not os.path.exists(path):
  os.mkdir(path)

layer=0

while layer in range(len(outputs['hidden_states'])):
  if not os.path.exists(path+'/layer_' + str(layer)):
    os.mkdir(path+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs['hidden_states'][layer])):
        tensor = outputs['hidden_states'][layer][example][0]
        tensors.append(tensor)
        label = [small_tokenized_dataset['test']['text'][example],str(small_tokenized_dataset['test']['label'][example])]
        labels.append(label)
        example +=1

  writer=SummaryWriter(path+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['text','label'])

  layer+=1


worst epoch

In [ ]:
# worst epoch

fine_tuned_model_worstep = RobertaForSequenceClassification.from_pretrained("sample_cl_trainer_rob_man/checkpoint-40", num_labels=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

fine_tuned_model_worstep.to(device)

model_inputs = tokenizer(small_tokenized_dataset['test']['text'], padding=True, truncation=True, return_tensors='pt')
model_inputs = {key: value.to(device) for key, value in model_inputs.items()}

outputs = fine_tuned_model_worstep(**model_inputs, output_hidden_states=True)


In [ ]:
# worst epoch

path_worstep = "/content/drive/MyDrive/Colab Notebooks/Comp Ling/Project_Irony_Detection/roBERTa_results_vis_man_640_worstep_cp40"
if not os.path.exists(path_worstep):
  os.mkdir(path_worstep)

layer=0

while layer in range(len(outputs['hidden_states'])):
  if not os.path.exists(path_worstep+'/layer_' + str(layer)):
    os.mkdir(path_worstep+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs['hidden_states'][layer])):
        tensor = outputs['hidden_states'][layer][example][0] # [sp_token_position]
        tensors.append(tensor)
        label = [small_tokenized_dataset['test']['text'][example],str(small_tokenized_dataset['test']['label'][example])]
        labels.append(label)
        example +=1

  writer=SummaryWriter(path_worstep+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['text','label'])

  layer+=1
